## Laboratorio #5 Analisis de paquetes de Red


In [1]:
# Lo primero es importar las librerias que necesitamos 

from scapy.all import * # Packet manipulation
import pandas as pd # Pandas - Create and Manipulate DataFrames
import numpy as np # Math Stuff (don't worry only used for one line :] )
import binascii # Binary to Ascii 
import seaborn as sns
sns.set(color_codes=True)

In [4]:
# Ahora capturaremos 10 paquetes

num_of_packets_to_sniff = 10
pcap = sniff(count=num_of_packets_to_sniff)

# rdpcap returns packet list
## packetlist object can be enumerated 
print(type(pcap))
print(len(pcap))
print(pcap)
pcap[0]


<class 'scapy.plist.PacketList'>
10
<Sniffed: TCP:7 UDP:3 ICMP:0 Other:0>


<Ether  dst=9c:b6:d0:19:0b:9f src=74:3a:ef:4a:04:57 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=40 id=1053 flags=DF frag=0 ttl=111 proto=tcp chksum=0xe8a8 src=20.42.73.25 dst=192.168.1.31 |<TCP  sport=https dport=54923 seq=2869743664 ack=2912439723 dataofs=5 reserved=0 flags=A window=2053 chksum=0x39fd urgptr=0 |<Padding  load='\x00\x00\x00\x00\x00\x00' |>>>>

In [6]:
a = rdpcap('./analisis_paquetes.pcap')
print(a)

<analisis_paquetes.pcap: TCP:0 UDP:62 ICMP:0 Other:0>


In [8]:
# Ahora uniremos ambos paquetes y procederemos a trabajar con uno solo. 

pac1= pcap 
pac2 = a 

all = pac1 + pac2

wrpcap('combinado.pcap', all)
combi = rdpcap('./combinado.pcap')
print(combi)

<combinado.pcap: TCP:7 UDP:65 ICMP:0 Other:0>


Como podemos ver, ahora ya tenemmos un archivo que cuenta con todos los daros que necesitamos para trabajar, para eso ahora haremos un DataFrame con el archivo. 

In [10]:
# crear un dataframe del archivo: 

# Obtenemos los campos de IP/TCP/UDP que seran las columnas de nuestro DF
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

#Concatenamos todos los campos
dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

# Creamos un DF en blanco con los campos como columnas
df = pd.DataFrame(columns=dataframe_fields)
for packet in pcap[IP]:
    # Obtenemos los valores de los campos de cada fila
    field_values = []
    # A;adimos todos los valores de los campos de IP al df
    for field in ip_fields:
        if field == 'options':
            # Obtenemos el numero de opciones del IP Header
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    
    field_values.append(packet.time)
    
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    
    # Agregamos el payload
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    # Agregamos la fila al df
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)

# Reset Index
df = df.reset_index()
# Drop old index column
df = df.drop(columns="index")

print(df.head(6))

  version ihl tos   len     id flags frag  ttl proto chksum  ... dataofs  \
0       4   5   0    40   1053    DF    0  111     6  59560  ...       5   
1       4   5   0    40   1054    DF    0  111     6  59559  ...       5   
2       4   5   0  1119   1055    DF    0  111     6  58479  ...       5   
3       4   5   0    40  63437    DF    0   64     6   9208  ...       5   
4       4   5   0    71  63438    DF    0   64     6   9176  ...       5   
5       4   5   0    40  63439    DF    0   64     6   9206  ...       5   

  reserved flags  window chksum urgptr options payload  \
0        0     A    2053  14845      0    None       6   
1        0     A    2053  12015      0    None       6   
2        0   FPA    2053  46294      0       0    1079   
3        0     A     513  12475      0    None       0   
4        0    PA     513  20648      0       0      31   
5        0    FA     513  12443      0    None       0   

                                         payload_raw  \
0   

Como podemos ya tenemos armado el dataset, con todos los datos que necesitabamos, ahora procederemos a ubicar ciertos valores especificos. 

In [11]:
df[['src','dst','sport','dport']]

,src,dst,sport,dport
0,20.42.73.25,192.168.1.31,443,54923
1,20.42.73.25,192.168.1.31,443,54923
2,20.42.73.25,192.168.1.31,443,54923
3,192.168.1.31,20.42.73.25,54923,443
4,192.168.1.31,20.42.73.25,54923,443
5,192.168.1.31,20.42.73.25,54923,443
6,20.42.73.25,192.168.1.31,443,54923
7,192.168.1.25,255.255.255.255,56045,6667
8,192.168.1.18,255.255.255.255,55098,6667
9,192.168.1.31,216.230.147.90,65383,53


In [12]:
#Calcular estadisticas
print("# Top Source Address")
print(df['src'].describe(),'\n\n')

# Top Source Address
count              10
unique              4
top       20.42.73.25
freq                4
Name: src, dtype: object 


